In [1]:
from load_data import parameters_dict, simulation_file_paths

import numpy as np
# import saved feature array
feature_arr = np.load("feature_arr.npy")

In [2]:
# check gpu
import tensorflow as tf
print("Num GPUs Available: ", len(
    tf.config.experimental.list_physical_devices('GPU')))

2024-03-27 10:58:18.769527: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 10:58:19.430998: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2024-03-27 10:58:19.920589: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-27 10:58:19.947678: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-27 10:58:19.947982: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
from sklearn.cluster import DBSCAN


# function that returns the std of the distance between the centroids of the clusters
def std_distance_centroids(feature_arr, cluster_labels):
    centroids =[] 
    # fill the array with the centroids
    for i in cluster_labels:
        centroids.append(np.mean(feature_arr[cluster_labels == i], axis=0))
    # initialize the array that will contain the distances
    distances =[] 
    # fill the array with the distances
    for i in range(len(centroids)):
        for j in range(len(centroids)):
            distances.append(np.linalg.norm(centroids[i] - centroids[j]))
    # return the std of the distances
    return np.std(distances)

In [4]:
import optuna


def objective(trial):
    eps = trial.suggest_float("eps", 0.5, 1.5)
    min_samples = trial.suggest_int("min_samples", 2, 10)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    cluster_labels = dbscan.fit_predict(feature_arr)
    std_dist = std_distance_centroids(feature_arr, cluster_labels)
    return std_dist


study = optuna.create_study(direction="minimize", study_name="dbscan_centroids_min",
                            storage="sqlite:///test.db", load_if_exists=True)
study.optimize(objective, n_trials=0)

[I 2024-03-27 10:58:20,629] Using an existing study with name 'dbscan_centroids_min' instead of creating a new one.


In [13]:
dbscan = DBSCAN(eps=study.best_params["eps"], min_samples=study.best_params["min_samples"])

dbscan.fit(feature_arr)
print(len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0))

import plotly.graph_objs as go
import plotly.express as ptx

class_dict = {f"Class {i}": list() for i in range(len(set(dbscan.labels_)))}
class_dict["Outliers"] = list()
for index, path in enumerate(simulation_file_paths):
    class_ = dbscan.labels_[index]
    if class_ == -1:
        class_dict["Outliers"].append(str(path))
    else:
        class_dict[f"Class {class_}"].append(str(path))

traces = list()
for class_id, colour in zip(class_dict, ptx.colors.qualitative.Dark24):
    traces.append(
        go.Scatter(
            x=[
                parameters_dict[f]["H"]
                for f in class_dict[class_id]
            ],
            y=[
                parameters_dict[f]["E"]
                for f in class_dict[class_id]
            ],
            mode="markers",
            name=class_id,
            opacity=0.7,
            marker=dict(
                size=11,
                color=colour,
            ),
        )
    )

fig = go.Figure(data=traces)
fig.update_layout(
    autosize=False,
    width=990,
    height=600,
    margin=dict(
        l=10,
        r=10,
        b=25,
        t=25,
    ),
    xaxis_title=r"$\mu_0 \mathbf{H} \text{ (T)}$",
    yaxis_title=r"$\Delta \text{E (J)}$",
)

25


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#2E91E5', 'size': 11},
              'mode': 'markers',
              'name': 'Class 0',
              'opacity': 0.7,
              'type': 'scatter',
              'x': [0.6698909203044445, 0.6698909203044445, 0.6698909203044445,
                    ..., 0.6546227281234838, 0.6546227281234838,
                    0.6546227281234838],
              'y': [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]},
             {'marker': {'color': '#E15F99', 'size': 11},
              'mode': 'markers',
              'name': 'Class 1',
              'opacity': 0.7,
              'type': 'scatter',
              'x': [0.6698909203044445, 0.7108584663581602, 1.0250054051919253,
                    1.0250054051919253, 1.0520917899467686, 0.9646048372886914,
                    0.9646048372886914, 0.9207083700108073, 0.9207083700108073,
                    0.7015080933424875, 1.1929253724128137, 0.746403496136002,
                    1.0976181620612193, 1.1469998419153746, 0.9808002518152286,
                    0.9808002518152286, 0.6597066901473676, 0.9291651296158059,
                    0.9291651296158059, 0.7125494922138771, 0.9249011197760583,
                    0.9249011197760583, 0.836415867916509, 0.720271961003612,
                    1.1141259767768092, 1.1071872073552032, 1.1086917079049514,
                    1.1955561140906563, 0.8202076505365383, 0.8202076505365383,
                    0.9864011704620863, 0.9864011704620863, 0.7453627241490277,
                    1.1305127224462546, 0.657065121899311, 0.8197104804685906,
                    0.8197104804685906, 1.0847157664043714, 0.7867600810417402,
                    1.0202856143980115, 1.0202856143980115, 0.8028197934355251,
                    0.8028197934355251, 1.1954039370060294, 0.8347188180131706,
                    1.0138643716232256, 1.0138643716232256, 0.8362432922424539,
                    0.9066727290485987, 0.9066727290485987, 0.7520385821733719,
                    0.8775627476181801, 0.8775627476181801, 1.1735348343250152,
                    0.8115632219784964, 0.8115632219784964, 0.7038115819871134,
                    1.1827621140750701, 0.9130732008672463, 0.9130732008672463,
                    0.6699039315395161, 1.1661932580993006, 0.6873601795375328,
                    0.6570880888695172, 1.1090159870748109, 0.790605703246432,
                    0.9269515687798381, 0.9269515687798381, 0.9698279207025753,
                    0.9698279207025753, 1.1770707825786892, 1.116615272717344,
                    1.0718182521919446, 0.8843678484873443, 0.8843678484873443,
                    0.9641370033042524, 0.9641370033042524, 1.0505179873860966,
                    0.8027673907963068, 0.8027673907963068, 1.1402152872098037,
                    0.6848079906719802, 0.7474323434833914, 0.8237846523064629,
                    0.9581704014515433, 0.9581704014515433, 1.0706344716555656,
                    0.7897525671244984, 0.9935376695424457, 0.9935376695424457,
                    0.8850705377887228, 0.8850705377887228, 0.9742161584909454,
                    0.9742161584909454, 0.6997767539785071, 0.7983981074624066,
                    1.173866898580667, 0.6742951967549535, 0.8204690678947646,
                    0.8204690678947646, 1.0934684341269973, 1.0515676079188805,
                    0.7179679094332169, 0.9898962581019028, 0.9898962581019028,
                    1.112307266850654, 1.140888569640036, 1.0282460428447024,
                    1.0282460428447024, 0.9981815390658192, 0.9981815390658192,
                    0.8637599795213791, 0.8637599795213791, 0.9448716255746837,
                    0.9448716255746837, 0.9953320671441747, 0.9953320671441747,
                    0.9951803409357836, 0.9951803409357836, 1.1603903092753214,
                    0.9861663886355949, 0.9861663886355949, 1.1120120774019429,
                    1.1122117803842337, 1.0465188935411227, 1.016188363672831,
                    1.01